# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [66]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [67]:
# your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [68]:
orders.columns= orders.columns.str.lower()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [69]:
# your code here
orders1 = orders.groupby(['customerid']).agg({'amount_spent':sum})
orders1

,amount_spent
customerid,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [70]:
def rating(x):
    if x>=orders1['amount_spent'].quantile(0.95):
        return 'VIP'
    if x>=orders1['amount_spent'].quantile(0.75):
        return 'Preferred'
    else:
        return 'Peanuts'

In [71]:
orders1['rating'] = orders1['amount_spent'].apply(rating)

In [72]:
orders1['rating'].value_counts()

Peanuts      3254
Preferred     868
VIP           217
Name: rating, dtype: int64

In [73]:
orders1

,amount_spent,rating
customerid,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Peanuts
...,...,...
18280,180.60,Peanuts
18281,80.82,Peanuts
18282,178.05,Peanuts


In [74]:
orders_r = pd.merge(left = orders,
                                 right = orders1,
                                 how = 'left', 
                                 left_on = "customerid", 
                                 right_on= "customerid")
display(orders_r.shape)
orders_r.head()

(397924, 16)

,unnamed: 0,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent_x,amount_spent_y,rating
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [75]:
orders2 = orders.groupby(['country', 'customerid']).agg({'amount_spent':sum})
orders2

amount_spent
country        customerid              
Australia      12386             401.90
               12388            2780.66
               12393            1582.60
               12415          124914.53
               12422             386.20
...                                 ...
United Kingdom 18287            1837.28
Unspecified    12363             552.00
               12743             546.43
               14265             530.18
               16320            1038.46

[4347 rows x 1 columns]

In [76]:
orders2['rating'] = orders2['amount_spent'].apply(rating)

In [77]:
orders2['rating'].value_counts()

Peanuts      3262
Preferred     869
VIP           216
Name: rating, dtype: int64

In [78]:
orders_r = pd.merge(left = orders,
                                 right = orders2,
                                 how = 'left', 
                                 left_on = "customerid", 
                                 right_on= "customerid")
display(orders_r.shape)
orders_r.head()

(398835, 16)

,unnamed: 0,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent_x,amount_spent_y,rating
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


In [79]:
# your code here
vips = orders_r.groupby(['country', 'customerid']).agg({'rating':max})
result = []
for x in vips['rating']:
    if  x == 'VIP':
        result.append(1)
    else:
        result.append(0)
vips['vip'] = result
vips = vips.groupby(['country']).agg({'vip':sum})
#vips = vips['vip'].idxmax()
vips

,vip
country,
Australia,1
Austria,0
Bahrain,0
Belgium,0
Brazil,0
Canada,0
Channel Islands,1
Cyprus,1
Czech Republic,0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [80]:
# your code here
vip_pref = orders_r.groupby(['country', 'customerid']).agg({'rating':max})
result1 = []
for x in vip_pref['rating']:
    if  x == 'VIP':
        result1.append(1)
    elif x ==  'Preferred':
        result1.append(1)
    else:
        result1.append(0)
vip_pref['vip_pref'] = result1
vip_pref = vip_pref.groupby(['country']).agg({'vip_pref':sum})
#vip_pref = vip_pref['vip'].idxmax()
vip_pref

,vip_pref
country,
Australia,4
Austria,3
Bahrain,0
Belgium,12
Brazil,0
Canada,1
Channel Islands,4
Cyprus,4
Czech Republic,0
